In [ ]:
%store -r

In [ ]:
import boto3
import botocore.exceptions
import time

In [ ]:
# Get event source mapppings UUID

lambda_client = boto3.client('lambda')

try:
    list_event_source_mappings_response = lambda_client.list_event_source_mappings(
        FunctionName = LambdaFunctionName,
        EventSourceArn=MSKClusterArn
    )
    for mapping in list_event_source_mappings_response['EventSourceMappings']:
        print('Event source mapping UUID:', mapping['UUID'])
        event_source_mapping_uuid = mapping['UUID']
except botocore.exceptions.ClientError as error:
    print(error)
    raise error

In [ ]:
# Delete the event source mapping

lambda_client = boto3.client('lambda')

try:
    delete_event_source_mapping_response = lambda_client.delete_event_source_mapping(
        UUID = event_source_mapping_uuid
    )
    print(delete_event_source_mapping_response)
except botocore.exceptions.ClientError as error:
    print(error)
    raise error

In [ ]:
# Ensure the event source mapping id deleted

lambda_client = boto3.client('lambda')
mapping_check = 'active'

max_time = time.time() + 1*60*60 # 1 hour

while time.time() < max_time:
    try:
        list_event_source_mappings_response = lambda_client.list_event_source_mappings(
            FunctionName = LambdaFunctionName,
            EventSourceArn=MSKClusterArn
        )
        if not list_event_source_mappings_response['EventSourceMappings']:
            mapping_check = 'inactive'
        else:
            mapping_check = 'active'
    except botocore.exceptions.ClientError as error:
        print(error)
        raise error

    if mapping_check == "inactive":
        print('Event source mapping deleted')
        break
    else:
        print('Event source mapping deleting')
    time.sleep(30)